# Shortest Path Problem with Travel Time Uncertainty

In this lecture, we will explore the shortest path problem on a 5×5 grid with 25 nodes and 40 arcs. The unique aspect of this problem is that the travel times are uncertain. However, we have access to historical data spanning the last 100 days, which provides valuable insights into the travel times under various conditions such as weather and traffic.

Throughout this notebook, we will leverage this historical data to develop strategies for finding the shortest path while considering the uncertainty in travel times. By the end, you will have a solid understanding of how to use historical data to tackle optimization problem in scenarios where some level of uncertainty is present.

In [ ]:
# Install necessary libraries

!pip install networkx  # Used to construct the graph
!pip install numpy pandas scikit-learn
!pip install gurobipy pyomo

## Constructing the Graph

The grid graph has each node connected to its nearest adjacent neighbors. The following code creates a 5x5 grid with 25 nodes and 40 edges. The nodes are labeled as $(i, j)$, where $i$ and $j$ are integers ranging from 0 to $M-1$. The edges represent the connections between adjacent nodes in the grid. This graph can be used to model various optimization problems on a grid, such as the shortest path problem.


In [ ]:
import networkx as nx

# Create a grid graph with M nodes on each edge
M = 5
G = nx.grid_2d_graph(M, M)

# Draw the grid graph with node labels
nx.draw(G, with_labels=True)

In [ ]:
# Define the set of nodes and arcs
N = sorted(list(G.nodes()))
A = sorted(list(G.edges()))

print("First 10 nodes:", N[:10])
print("First 10 arcs:")
for arc in A[:10]:
    print(arc)

## Creating the Model

The shortest path problem is formulated as:

\begin{align*}
&\min \ && \sum_{(i,j) \in A} c_{ij}\, x_{ij} \\
&\text{subject to:}\ && \sum_{j: (i, j)\in A} x_{ij} = 1,\quad && i=O\\
    & && \sum_{i: (i, j)\in A} x_{ij} = 1,\quad && j=D\\
    & && \sum_{j: (i, j)\in A} x_{ij} = \sum_{j: (j, i)\in A} x_{ji},\quad && i\in N\setminus\{O, D\}\\
    & && x_{ij} \in \{0, 1\}, \quad && i, j\in N
\end{align*}


In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

solver_options = {}  # You can provide your WSL license information here
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, solver_options=solver_options)

In [ ]:
# Define origin and destination
O = N[0]
D = N[-1]

print(O, D)

In [ ]:
# Define a function that solves the shortest path problem for a given cost matrix
def solve_sp(costs):
    """
    Solves the shortest path problem using a binary optimization model.

    Parameters:
    - costs: A list containing the costs of each edge in the graph.

    Returns:
    - The objective value of the shortest path.
    - The selected arcs representing the shortest path.
    """

    mod = pyo.ConcreteModel()
    mod.x = pyo.Var(A, domain=pyo.Binary)

    mod.obj = pyo.Objective(
        expr=sum(costs[i] * mod.x[a] for i, a in enumerate(A)),
        sense=pyo.minimize,
    )

    mod.cons = pyo.ConstraintList()
    for i in N:
        if i == O:
            mod.cons.add(sum(mod.x[a] for a in A if a[0] == i) == 1)
        elif i == D:
            mod.cons.add(sum(mod.x[a] for a in A if a[1] == i) == 1)
        else:
            expr1 = sum(mod.x[a] for a in A if a[0] == i)
            expr2 = sum(mod.x[a] for a in A if a[1] == i)
            mod.cons.add(expr1 == expr2)

    # Solve the model
    opt.solve(mod)

    # Get the objective value
    obj = pyo.value(mod.obj)

    # Get the selected arcs
    sol = [pyo.value(mod.x[a]) for a in A]

    return obj, sol

## Reading the Historical Data

To begin our analysis, we need to read the historical data that contains information about travel times under various conditions. This data spans the last 100 days and will be used to develop strategies for finding the shortest path while considering the uncertainty in travel times.

In [ ]:
import pandas as pd

# Read the historical data
hist = pd.read_csv("hist_data_100.csv", index_col=0)

hist.head()

## Training the Machine Learning Model

### Create Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# find the number of features
F = hist.shape[1] - len(A)
F

In [ ]:
# Define independent and dependent variables
X = hist.iloc[:, -F:].values
Y = hist.iloc[:, :-F].values

print("Features:")
print(X[0])
print("\nCosts:")
print(Y[0])

In [ ]:
from sklearn.linear_model import LinearRegression

# Split the data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2024)

# Run a linear regression model to predict the arc costs
ml = LinearRegression()
ml.fit(X_train, Y_train)

In [ ]:
# Predict the arc costs
Y_pred = ml.predict(X_test)

# What is the R^2 score of this model?
ml.score(X_test, Y_test)

## Solve Shortest Path Problems with the Predicted and Actual Costs on the Test Dataset

In [ ]:
def calc_total_cost(cost_matrix):
    total_costs = []
    solutions = []
    # iterate over all observations
    for cost_row in cost_matrix:
        # calculate the cost of the current observation
        obj_val, sol = solve_sp(cost_row)

        total_costs.append(obj_val)
        solutions.append(sol)

    return total_costs, solutions

In [ ]:
total_costs_pred, solution_pred = calc_total_cost(Y_pred)
total_costs_true, solution_true = calc_total_cost(Y_test)

### Find the Actual Total Costs of the Predicted Paths

In [ ]:
import numpy as np


def eval_solution(cost_matrix, solutions):
    total_costs = []
    # iterate over all solutions
    for i in range(len(cost_matrix)):
        c = cost_matrix[i]
        x = np.array(solutions[i])

        total_costs.append(np.dot(c, x))

    return total_costs


solution_costs = eval_solution(Y_test, solution_pred)

In [ ]:
print(total_costs_pred)
print(total_costs_true)
print(solution_costs)

### Calculate the Error

Compare the cost of the actual solutions on the test set with the actual costs of the predicted solutions.

In [ ]:
def calc_error(true_costs, predicted_costs):
    true_costs = np.array(true_costs)
    predicted_costs = np.array(predicted_costs)

    diffs = np.sum(np.abs(true_costs - predicted_costs))
    return diffs / np.sum(true_costs)


err = calc_error(total_costs_true, solution_costs)
err

## Expected Value Problem

In [ ]:
average_costs = np.mean(Y_test, axis=0)

average_costs

In [ ]:
obj_val_avg, sol_avg = solve_sp(average_costs)
# repeat solution for all observations
sol_avg = [sol_avg] * len(Y_test)

solution_costs_avg = eval_solution(Y_test, sol_avg)

err = calc_error(total_costs_true, solution_costs_avg)
err